# Imports

In [ ]:
 pip install plotly

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━ 15.0/15.7 MB 975.1 kB/s eta 0:00:01:01

In [1]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

/home/alex/.local/lib/python3.10/site-packages/sklearn/utils/__init__.py:16: UserWarning: A NumPy version >=1.22.4 and <1.29.0 is required for this version of SciPy (detected version 1.22.2)
  from scipy.sparse import issparse


ModuleNotFoundError: No module named 'plotly'

# Grabbing Data

In [ ]:
df_all = pd.read_pickle("../interactive/df_all_2005_2023.pkl")

In [ ]:
season = 2023
position = 'QB'

In [ ]:
df_review = df_all.query(f"season == {season} and position == '{position}' and games >= 5")

### Verifying Data

In [ ]:
assert(df_review["position"].unique()[0] == position)
assert(df_review["season"].unique()[0] == season)
assert(df_review["games"].max() == 17)

# Line Graph

In [ ]:
df_review.columns

In [ ]:
col_to_graph = "dakota"

In [ ]:
df_sorted = df_review.sort_values(by=col_to_graph, ascending=True).copy()

In [ ]:
per_game = True

In [ ]:
if per_game:
    df_sorted[f"{col_to_graph}_per_game"] = df_sorted[col_to_graph] / df_sorted["games"]
    col_to_graph += "_per_game"
    df_sorted.sort_values(by=col_to_graph, ascending=True, inplace=True)

In [ ]:
df_sorted.reset_index(inplace=True)

In [ ]:
# Calculate the reverse index for each row (starting from the bottom)
reverse_index = (len(df_sorted) - 1) - df_sorted.index

# Calculate the group for each row based on the reverse index
group = reverse_index // 12

In [ ]:
# Map each group to a color
colors = {0: 'Tier 1', 1: 'Tier 2', 2: 'Tier 3', 3: 'Tier 4', 4: 'Tier 5', 5: 'Tier 6'}
df_sorted['Color'] = group.map(colors)

In [ ]:
# scatter plot
start_index = -30
fig = px.scatter(df_sorted[start_index:], x="player_name", y=col_to_graph, color='Color')

# Add a horizontal line at qb1 line
qb1_line_h = df_sorted[col_to_graph].iloc[-12]
fig.add_shape(type="line",
              x0=0, x1=1, y0=qb1_line_h, y1=qb1_line_h,
              line=dict(color="black", width=2),
              xref="paper", yref="y")

# add a vertical line at qb1
qb1_line_v = df_sorted["player_name"].iloc[-12]
fig.add_shape(type="line",
              x0=qb1_line_v, x1=qb1_line_v, y0=df_sorted[col_to_graph].iloc[start_index - 10], 
              y1=df_sorted[col_to_graph].max()* 1.1,
              line=dict(color="black", width=2),
              xref="x", yref="y")

# Add a horizontal line at qb2 line
qb2_line_h = df_sorted[col_to_graph].iloc[-24]
fig.add_shape(type="line",
              x0=0, x1=1, y0=qb2_line_h, y1=qb2_line_h,
              line=dict(color="black", width=2),
              xref="paper", yref="y")

# add a vertical line at qb2
qb2_line_v = df_sorted["player_name"].iloc[-24]
fig.add_shape(type="line",
              x0=qb2_line_v, x1=qb2_line_v, y0=df_sorted[col_to_graph].iloc[start_index - 10], 
              y1=df_sorted[col_to_graph].max()* 1.1,
              line=dict(color="black", width=2),
              xref="x", yref="y")

fig.update_traces(marker=dict(size=12))

fig.update_layout(
    title='2023 QB Dakoa',
    xaxis_title="Player Names",
    yaxis_title='Dakota (Per Game)',
    legend_title='QB Tiers',
)
        
fig.write_html("../interactive-2.0/QB/2023-review-pt2/dakota-per-game.html")
fig.show()